<a href="https://colab.research.google.com/github/eeluedke/573/blob/main/notebooks/BioLay_EDA_SectionPresence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installs

In [2]:
!pip install datasets nltk textstat matplotlib seaborn readability-lxml spacy scikit-learn tqdm

import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import textstat
import spacy
from tqdm import tqdm
from datasets import load_dataset
from collections import Counter
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fs

True

In [3]:
!python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 47.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
plos = load_dataset("BioLaySumm/BioLaySumm2025-PLOS")
elife = load_dataset("BioLaySumm/BioLaySumm2025-eLife")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/693 [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/169M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/170M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/169M [00:00<?, ?B/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/6.92M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/3.27M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4346 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/241 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/142 [00:00<?, ? examples/s]

# Section Presence



In [5]:
plos

DatasetDict({
    train: Dataset({
        features: ['article', 'summary', 'section_headings', 'keywords', 'year', 'title'],
        num_rows: 24773
    })
    validation: Dataset({
        features: ['article', 'summary', 'section_headings', 'keywords', 'year', 'title'],
        num_rows: 1376
    })
    test: Dataset({
        features: ['article', 'summary', 'section_headings', 'keywords', 'year', 'title'],
        num_rows: 142
    })
})

In [16]:
plos['train'][0]['article'].split('\n')[4]

'Zebrafish were maintained and staged as described [80 , 81] . Tübingen strain wild-type embryos were used for all experiments . DEAB and all-trans retinoic acid ( Sigma-Aldrich ) were dissolved in 100% dimethyl sulfoxide ( DMSO ) to make a 1 M stock and aliquots were stored at −80°C . For DEAB and RA treatments: embryos were incubated in 1 . 6 × 10−5 M DEAB/DMSO in E3 embryo media , 1 × 10−6 M or 1 × 10−7 M RA/DMSO in E3 embryo media , or 1 . 6 × 10−5 M DMSO ( control ) in E3 in the dark over particular developmental intervals , then washed five times with E3 and then fixed at 24 or 48 hpf . These experimental treatments were fully penetrant and produced consistent results at the doses and treatment windows that were examined . raldh2 morpholino ( CAACTTCACTGGAGGTCATCGCGTC ) was injected into 1-cell wild-type embryos . Incrosses of kggtv205 heterozygous adults ( maintained on the Tübingen strain ) were used to obtain cdx4–/– embryos and were injected at the 1-cell stage with cdx1a mor

In [ ]:
def

In [ ]:
def check_section_consistency(dataset):
    """
    Checks for the presence of specific sections and their consistency across documents in a dataset.
    """
    section_counts = Counter()
    for doc in tqdm(dataset, desc="Analyzing sections"):  # tqdm for progress bar
        for section in doc["section_headings"]:
            section = section.lower()  # Convert to lowercase for consistency
            section_counts[section] += 1
    print("Section consistency:")
    for section, count in section_counts.items():
        print(f"  {section}: {count}/{len(dataset)} ({count / len(dataset) * 100:.2f}%)")

# Execute the function on each dataset:
check_section_consistency(plos["train"])  # For PLOS dataset, assuming 'sections' feature
check_section_consistency(elife["train"])  # For eLife dataset, assuming 'sections' feature

Analyzing sections: 100%|██████████| 24773/24773 [00:05<00:00, 4320.19it/s]


Section consistency:
  abstract: 24773/24773 (100.00%)
  introduction: 24736/24773 (99.85%)
  results: 23696/24773 (95.65%)
  discussion: 23740/24773 (95.83%)
  materials: 16589/24773 (66.96%)
  and: 17811/24773 (71.90%)
  methods: 24207/24773 (97.72%)
  supporting: 351/24773 (1.42%)
  information: 351/24773 (1.42%)
  results/discussion: 875/24773 (3.53%)
  material: 364/24773 (1.47%)
  models: 194/24773 (0.78%)
  &: 71/24773 (0.29%)
  rule: 20/24773 (0.08%)
  1:: 4/24773 (0.02%)
  use: 11/24773 (0.04%)
  version: 4/24773 (0.02%)
  control: 7/24773 (0.03%)
  2:: 5/24773 (0.02%)
  document: 1/24773 (0.00%)
  your: 6/24773 (0.02%)
  code: 3/24773 (0.01%)
  usage: 1/24773 (0.00%)
  3:: 4/24773 (0.02%)
  make: 2/24773 (0.01%)
  common: 2/24773 (0.01%)
  operations: 1/24773 (0.00%)
  easy: 1/24773 (0.00%)
  to: 21/24773 (0.08%)
  4:: 2/24773 (0.01%)
  releases: 3/24773 (0.01%)
  5:: 2/24773 (0.01%)
  reuse: 1/24773 (0.00%)
  software: 9/24773 (0.04%)
  (within: 1/24773 (0.00%)
  reason): 1/

Analyzing sections: 100%|██████████| 4346/4346 [00:00<00:00, 5687.20it/s]

Section consistency:
  abstract: 4346/4346 (100.00%)
  introduction: 4317/4346 (99.33%)
  results: 4279/4346 (98.46%)
  discussion: 4286/4346 (98.62%)
  materials: 4169/4346 (95.93%)
  and: 4523/4346 (104.07%)
  methods: 4268/4346 (98.21%)
  material: 83/4346 (1.91%)
  method: 1/4346 (0.02%)
  main: 10/4346 (0.23%)
  text: 9/4346 (0.21%)
  ethics: 1/4346 (0.02%)
  statement: 1/4346 (0.02%)
  acknowledgments: 5/4346 (0.12%)
  smoothness: 1/4346 (0.02%)
  the: 5/4346 (0.12%)
  neural: 1/4346 (0.02%)
  code: 1/4346 (0.02%)
  model: 10/4346 (0.23%)
  deep: 1/4346 (0.02%)
  learning: 1/4346 (0.02%)
  networks: 1/4346 (0.02%)
  mathematical: 2/4346 (0.05%)
  experimental: 2/4346 (0.05%)
  procedures: 1/4346 (0.02%)
  modeling: 1/4346 (0.02%)
  framework: 2/4346 (0.05%)
  accession: 3/4346 (0.07%)
  numbers: 2/4346 (0.05%)
  set-up: 1/4346 (0.02%)
  of: 7/4346 (0.16%)
  quorum-sensing: 1/4346 (0.02%)
  numerical: 1/4346 (0.02%)
  simulations: 1/4346 (0.02%)
  analysis: 3/4346 (0.07%)
  clinic

# Term Frequency Analysis


In [ ]:
plos

DatasetDict({
    train: Dataset({
        features: ['article', 'summary', 'section_headings', 'keywords', 'year', 'title'],
        num_rows: 24773
    })
    validation: Dataset({
        features: ['article', 'summary', 'section_headings', 'keywords', 'year', 'title'],
        num_rows: 1376
    })
    test: Dataset({
        features: ['article', 'summary', 'section_headings', 'keywords', 'year', 'title'],
        num_rows: 142
    })
})

In [ ]:
def extract_text(dataset):
    """
    Extracts full text and lay summaries from a dataset.
    """
    full_texts = [doc['article'] for doc in dataset]
    lay_summaries = [doc['summary'] for doc in dataset]
    return full_texts, lay_summaries

# Extract text from both datasets:
plos_full_texts, plos_lay_summaries = extract_text(plos["train"])
elife_full_texts, elife_lay_summaries = extract_text(elife["train"])

In [ ]:
def create_term_frequency(texts):
    """
    Creates a term frequency distribution using CountVectorizer.
    """
    vectorizer = CountVectorizer(stop_words="english")  # Remove common English words
    term_frequency = vectorizer.fit_transform(texts)
    return vectorizer, term_frequency

# Create term frequency distributions:
plos_full_vectorizer, plos_full_freq = create_term_frequency(plos_full_texts)
plos_lay_vectorizer, plos_lay_freq = create_term_frequency(plos_lay_summaries)
elife_full_vectorizer, elife_full_freq = create_term_frequency(elife_full_texts)
elife_lay_vectorizer, elife_lay_freq = create_term_frequency(elife_lay_summaries)

In [ ]:
def identify_technical_terms(full_vectorizer, full_freq, lay_vectorizer, lay_freq, top_n=100):
    """
    Identifies technical terms by comparing term frequencies.
    """
    full_terms = full_vectorizer.get_feature_names_out()
    lay_terms = lay_vectorizer.get_feature_names_out()
    full_term_freq = np.asarray(full_freq.sum(axis=0)).ravel()  # Sum frequencies across documents
    lay_term_freq = np.asarray(lay_freq.sum(axis=0)).ravel()

    technical_terms = []
    for term, full_count in zip(full_terms, full_term_freq):
        if term in lay_terms:
            lay_count = lay_term_freq[lay_terms.tolist().index(term)]
            if full_count > 10 and lay_count < 5:  # Adjust thresholds as needed
                technical_terms.append(term)
    return technical_terms[:top_n]  # Return top N terms

# Identify technical terms for both datasets:
plos_technical_terms = identify_technical_terms(plos_full_vectorizer, plos_full_freq, plos_lay_vectorizer, plos_lay_freq)
elife_technical_terms = identify_technical_terms(elife_full_vectorizer, elife_full_freq, elife_lay_vectorizer, elife_lay_freq)
print("PLOS Technical Terms:", plos_technical_terms)
print("eLife Technical Terms:", elife_technical_terms)

PLOS Technical Terms: ['00', '0003', '0005', '000mf', '002', '00201', '004', '005', '007', '008', '009', '010', '011', '012', '013', '0139', '015', '017', '018', '019', '0201', '021', '023', '024', '025', '0258', '026', '028', '034', '035', '036', '037', '039', '040', '041', '042', '045', '046', '047', '050', '051', '052', '053', '055', '056', '057', '058', '061', '063', '06446846', '066', '067', '071', '077', '080', '0802', '081', '086', '089', '090', '0901', '091', '092', '093', '094', '095', '096', '097', '10000', '100000', '1000gp', '1000s', '10074', '100kb', '100m', '100mm', '100nm', '100s', '100μm', '1010', '1011', '1013', '1016', '1017', '1018', '1024', '1026b', '102l', '1031', '106b', '1080', '1082', '10a', '10b', '10d3', '10hz', '10kb', '10kda', '10ms', '10nos2']
eLife Technical Terms: ['002', '016', '019', '036', '041', '069', '073', '09594', '10000', '1000s', '101', '102', '103', '105', '10640', '1071', '109', '110', '1106', '112', '113', '117', '118', '122', '124', '125', '